In [1]:
import numpy as np
import pandas as pd
import re
pd.set_option('display.float_format', lambda x : '%.30f' % x)
pd.options.display.max_columns = None
pd.options.display.float_format = "{:,.3f}".format
import warnings
warnings.filterwarnings('ignore')

In [2]:
dataset = pd.read_excel("A-2 General Ledger report to 31.12.2021.xls",header = None)

In [3]:
a2 = dataset.iloc[5:-4,:]
a2.iloc[:,0] = a2.iloc[:,0].str.strip()

In [4]:
len(a2[0].unique())

198

In [5]:
drop_inde = a2.loc[a2[0].str.startswith("Total") == True].index
len(drop_inde)

99

In [6]:
a2.drop(drop_inde,axis = 0,inplace = True)

In [7]:
a2[0].loc[a2[0].str.startswith("Total") == True]

Series([], Name: 0, dtype: object)

In [8]:
len(a2)

22868

In [9]:
col = list(dataset.iloc[4,:])
col

['Account ',
 'Date',
 'Transaction Type',
 'No.',
 'Name',
 'Memo/Description',
 'Split',
 'Amount',
 'Balance']

In [10]:
a2.columns = col

In [11]:
a2["Account "]
a2["Account "].isnull

<bound method Series.isnull of 5        Barter Account
6                   NaN
7                   NaN
8                   NaN
9                   NaN
              ...      
22967               NaN
22968               NaN
22969               NaN
22970               NaN
22971               NaN
Name: Account , Length: 22868, dtype: object>

In [12]:
len(a2.iloc[:,0].unique())
a2.iloc[:,0].unique()
a2.reset_index(drop = True)

,Account,Date,Transaction Type,No.,Name,Memo/Description,Split,Amount,Balance
0,Barter Account,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,04/01/2021,Payment,NaN,Website Sale,NaN,Debtors,3.150,3.150
2,NaN,04/01/2021,Payment,NaN,Website Sale,NaN,Debtors,6.220,9.370
3,NaN,05/01/2021,Payment,NaN,Website Sale,NaN,Debtors,5.510,14.880
4,NaN,06/01/2021,Payment,NaN,Website Sale,NaN,Debtors,3.830,18.710
...,...,...,...,...,...,...,...,...,...
22863,NaN,31/12/2021,Bill Payment (Cheque),Maria2188,Field Sports Publishing Inc,NaN,Creditors,0,0
22864,NaN,31/12/2021,Bill Payment (Cheque),Maria2192,Irons Brothers Ltd,NaN,Creditors,0,0
22865,NaN,31/12/2021,Bill Payment (Cheque),Maria2188,Reimer Johannsen,NaN,Creditors,0,0
22866,NaN,31/12/2021,Bill Payment (Cheque),Maria2188,Wbc,NaN,Creditors,0,0


In [13]:
a2["Account "].loc[a2["Account "].isnull() == True] = "-"
a2

,Account,Date,Transaction Type,No.,Name,Memo/Description,Split,Amount,Balance
5,Barter Account,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-,04/01/2021,Payment,NaN,Website Sale,NaN,Debtors,3.150,3.150
7,-,04/01/2021,Payment,NaN,Website Sale,NaN,Debtors,6.220,9.370
8,-,05/01/2021,Payment,NaN,Website Sale,NaN,Debtors,5.510,14.880
9,-,06/01/2021,Payment,NaN,Website Sale,NaN,Debtors,3.830,18.710
...,...,...,...,...,...,...,...,...,...
22967,-,31/12/2021,Bill Payment (Cheque),Maria2188,Field Sports Publishing Inc,NaN,Creditors,0,0
22968,-,31/12/2021,Bill Payment (Cheque),Maria2192,Irons Brothers Ltd,NaN,Creditors,0,0
22969,-,31/12/2021,Bill Payment (Cheque),Maria2188,Reimer Johannsen,NaN,Creditors,0,0
22970,-,31/12/2021,Bill Payment (Cheque),Maria2188,Wbc,NaN,Creditors,0,0


In [14]:
account = list(a2.iloc[:,0].unique())

In [15]:
account.remove("-")

In [16]:
account

['Barter Account',
 'Cash in Hand',
 'Commercial card',
 'Commerzbank',
 'Nat West 5869',
 'NatWest ****7810',
 'Natwest Saver ****7829',
 'Petty Cash',
 'Debtors',
 'Long Term Investment  (TEG)',
 'Prepayments',
 'Rising bite project',
 'Stock / Inventory Asset',
 'Stock in Trnasit',
 'Work in Progress',
 'Accumulated Depreciation',
 'Furniture and Fixtures',
 'Guns & Rifles - Museum',
 'Machinery',
 'Museum Items',
 'Office equipment',
 'Research and Development',
 'Vehicles',
 'Web development',
 'Creditors',
 'Accruals',
 'Corporation tax',
 'Deferred Tax',
 'Net Pay',
 'Orders in Hand/Deferred Income',
 'PAYE Tax',
 'Pension Payable',
 'VAT Control',
 'VAT Suspense',
 'Blaser Group Loan',
 'Blaser Loan',
 'Dividend',
 'Retained Earnings',
 'Share Capital',
 'Discounts given',
 'Dividend Income',
 'Sales of Product Income',
 'Non UK Sales',
 'Sales',
 'UK Sales',
 'Commission',
 'Cost of sales',
 'Materials & Tools',
 'Shipping & Freight',
 'Storage',
 'Accountancy',
 'ADMINISTRATI

In [17]:
bank_count = -1
counter = 0 #22770
for i in a2["Date"]:
    if type(i) == float:
        bank_count+=1
        a2.iloc[counter,0] = account[bank_count]
        counter+=1
    else:
        a2.iloc[counter,0] = account[bank_count]
        counter+=1

In [18]:
a2

,Account,Date,Transaction Type,No.,Name,Memo/Description,Split,Amount,Balance
5,Barter Account,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Barter Account,04/01/2021,Payment,NaN,Website Sale,NaN,Debtors,3.150,3.150
7,Barter Account,04/01/2021,Payment,NaN,Website Sale,NaN,Debtors,6.220,9.370
8,Barter Account,05/01/2021,Payment,NaN,Website Sale,NaN,Debtors,5.510,14.880
9,Barter Account,06/01/2021,Payment,NaN,Website Sale,NaN,Debtors,3.830,18.710
...,...,...,...,...,...,...,...,...,...
22967,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2188,Field Sports Publishing Inc,NaN,Creditors,0,0
22968,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2192,Irons Brothers Ltd,NaN,Creditors,0,0
22969,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2188,Reimer Johannsen,NaN,Creditors,0,0
22970,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2188,Wbc,NaN,Creditors,0,0


In [19]:
drp = a2[a2["Date"].isnull()].index

In [20]:
a2.drop(drp,inplace=True)

In [21]:
a2.reset_index(drop = True)

,Account,Date,Transaction Type,No.,Name,Memo/Description,Split,Amount,Balance
0,Barter Account,04/01/2021,Payment,NaN,Website Sale,NaN,Debtors,3.150,3.150
1,Barter Account,04/01/2021,Payment,NaN,Website Sale,NaN,Debtors,6.220,9.370
2,Barter Account,05/01/2021,Payment,NaN,Website Sale,NaN,Debtors,5.510,14.880
3,Barter Account,06/01/2021,Payment,NaN,Website Sale,NaN,Debtors,3.830,18.710
4,Barter Account,06/01/2021,Payment,NaN,Website Sale,NaN,Debtors,2.980,21.690
...,...,...,...,...,...,...,...,...,...
22765,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2188,Field Sports Publishing Inc,NaN,Creditors,0,0
22766,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2192,Irons Brothers Ltd,NaN,Creditors,0,0
22767,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2188,Reimer Johannsen,NaN,Creditors,0,0
22768,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2188,Wbc,NaN,Creditors,0,0


In [22]:
a2

,Account,Date,Transaction Type,No.,Name,Memo/Description,Split,Amount,Balance
6,Barter Account,04/01/2021,Payment,NaN,Website Sale,NaN,Debtors,3.150,3.150
7,Barter Account,04/01/2021,Payment,NaN,Website Sale,NaN,Debtors,6.220,9.370
8,Barter Account,05/01/2021,Payment,NaN,Website Sale,NaN,Debtors,5.510,14.880
9,Barter Account,06/01/2021,Payment,NaN,Website Sale,NaN,Debtors,3.830,18.710
10,Barter Account,06/01/2021,Payment,NaN,Website Sale,NaN,Debtors,2.980,21.690
...,...,...,...,...,...,...,...,...,...
22967,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2188,Field Sports Publishing Inc,NaN,Creditors,0,0
22968,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2192,Irons Brothers Ltd,NaN,Creditors,0,0
22969,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2188,Reimer Johannsen,NaN,Creditors,0,0
22970,Not Specified,31/12/2021,Bill Payment (Cheque),Maria2188,Wbc,NaN,Creditors,0,0


In [23]:
a2.to_excel("dump.xls", index = False)

PermissionError: [Errno 13] Permission denied: 'dump.xls'

In [ ]:
a2